In [ ]:
import pandas as pd
air_data = pd.read_csv("air_data.csv",sep='\t')

In [ ]:
air_data.head()

In [ ]:
def get_airport(row):
    if row['pickup_distance_to_jfk'] < 1000:
        return "JFK"
    elif row['pickup_distance_to_lga'] < 1000:
        return "LGA"
    elif row['pickup_distance_to_ewr'] < 1000:
        return "EWR"

In [ ]:
air_data['airport'] = air_data.apply(lambda row: get_airport(row), axis=1)

In [ ]:
air_data.to_csv("data_with_airports.csv", sep='\t')

In [ ]:
import csv
import json
import requests

# from .config import GOOGLE_API_KEY, MAPQUEST_API_KEY
MAPQUEST_API_KEY = 'SkYO2BUv0em7XMnplRvPbyu8AXwgjYTA'

MAPQUEST_DIRECTIONS = 'http://open.mapquestapi.com/directions/v2/route'

WRITE_KEYS = (
    'vendor', 'pickupTime', 'duration',
    'airport', 'direction',
)


def get_mapquest_polyline(value):
    value = json.loads(value)
    return value['route']['shape']['shapePoints']


def get_pickup_dropoff(data):
    return (
        '{},{}'.format(data['Pickup_latitude'], data['Pickup_longitude']),
        '{},{}'.format(data['Dropoff_latitude'], data['Dropoff_longitude'])
    )


def get_mapquest_direction(data):
    pickup, dropoff = get_pickup_dropoff(data)
    params = {
        'from': pickup, 'to': dropoff, 'key': MAPQUEST_API_KEY,
        'fullShape': 'true', 'shapeFormat': 'cmp', 'manMaps': 'false',
        'narrativeType': 'none', 'doReverseGeocode': 'false',
    }
    resp = requests.get(url=MAPQUEST_DIRECTIONS, params=params)
    if resp.status_code == 200:
        return get_mapquest_polyline(resp.content)
    return 'route-error'


def record_error(row):
    print(row)

def extract_data():
    for i in range(len(air_data)):
        row = air_data.iloc[i]
        data = {
            'vendor': row['VendorID'],
            'pickupTime': row['lpep_pickup_datetime'],
            'duration': row['trip_time_in_secs'],
            'airport': row['airport'],
            'direction': 'error',
        }
        try:
            data['direction'] = get_mapquest_direction(row)
        except:
            record_error(row)
        yield data
        
def write_with_directions():
    with open('with_directions.csv', 'w+') as f:
        writer = csv.DictWriter(f, WRITE_KEYS)
        rows = extract_data()
        for row in rows:
            writer.writerow(row)


if __name__ == '__main__':
    write_with_directions()